In [4]:
import numpy as np
import json
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from src.smpl.model import SMPLModel
from src.smpl.render import SMPLRender
from cv2 import Rodrigues
import pyrender
import trimesh
from tqdm.notebook import tqdm


In [5]:
import os
os.environ['PYOPENGL_PLATFORM'] = 'egl'

In [6]:
smpl_model = SMPLModel("models/smpl/basicmodel_neutral_lbs_10_207_0_v1.1.0.pkl")

In [7]:
width, height = 4032, 3040
path = "/data/teton_data/train/aeh/1421622085700_1/2023_05_20/2023_05_20_04_51_05_856_563/"
with open(os.path.join(path, "smpl_pseudo_gt.json"), "r") as f:
    data = json.load(f)


In [8]:
def create_raymond_lights():
    """
    Return raymond light nodes for the scene.
    """
    thetas = np.pi * np.array([1.0 / 6.0, 1.0 / 6.0, 1.0 / 6.0])
    phis = np.pi * np.array([0.0, 2.0 / 3.0, 4.0 / 3.0])

    nodes = []

    for phi, theta in zip(phis, thetas):
        xp = np.sin(theta) * np.cos(phi)
        yp = np.sin(theta) * np.sin(phi)
        zp = np.cos(theta)

        z = np.array([xp, yp, zp])
        z = z / np.linalg.norm(z)
        x = np.array([-z[1], z[0], 0.0])
        if np.linalg.norm(x) == 0:
            x = np.array([1.0, 0.0, 0.0])
        x = x / np.linalg.norm(x)
        y = np.cross(z, x)

        matrix = np.eye(4)
        matrix[:3,:3] = np.c_[x,y,z]
        nodes.append(pyrender.Node(
            light=pyrender.DirectionalLight(color=np.ones(3), intensity=1.0),
            matrix=matrix
        ))

    return nodes


In [13]:

renderer = pyrender.OffscreenRenderer(viewport_width=width, viewport_height=height, point_size=1.0)
scene = pyrender.Scene(bg_color=[0.0, 0.0, 0.0, 0.0], ambient_light=(0.3, 0.3, 0.3)) # Transparent background
camera_pose = np.eye(4)
focal_length_at_256 = 5000
focal_length = focal_length_at_256 / 256 * max(width, height)
camera = pyrender.IntrinsicsCamera(fx=focal_length, fy=focal_length, cx=width/2, cy=height/2, zfar=1e12)
camera_node = pyrender.Node(camera=camera, matrix=camera_pose)

light = pyrender.SpotLight(
    color=np.ones(3),
    intensity=3.0,
    innerConeAngle=np.pi / 16,
    outerConeAngle=np.pi / 6.0,
)
light_pose = np.eye(4)
light_node = pyrender.Node(light=light, matrix=light_pose)


light_nodes = create_raymond_lights()
for node in light_nodes: 
    scene.add_node(node)

In [17]:
i = 0

for frame_idx in tqdm(range(100)):
    frame_id = str(frame_idx)
    scene.clear()
    scene.add_node(camera_node)
    scene.add_node(light_node)

    img_path = os.path.join(path, "images", "{}.jpg".format(frame_idx))
    img = plt.imread(img_path)
    img = img.copy()

    for track_id, track_data in data.items():
        if frame_id not in track_data:
            # No data for this frame
            continue

        frame_data = track_data[frame_id]

        beta = np.array(frame_data["betas"])
        body_pose = np.array(frame_data['body_pose'])
        global_orient = np.array(frame_data['global_orient'])
        smpl_2d_kpts = np.array(frame_data["smpl_2d_kpts"])
        teton_2d_kpts = np.array(frame_data["teton_2d_kpts"])

        pred_cam = np.array(frame_data["pred_cam"])
        pred_cam_t_full = np.array(frame_data["pred_cam_t_full"])

        pose_Rs = np.vstack((global_orient, body_pose))

        pose = np.vstack([Rodrigues(R)[0] for R in pose_Rs])


        # Update the model and get the vertices
        v = smpl_model.set_params(beta=beta, pose=pose)
        v_translated = v + pred_cam_t_full

        mesh = trimesh.Trimesh(vertices=v_translated, faces=smpl_model.faces)
        # Apparently, the code in `render.MTL_yolov7/utils/renderer.py` flips the mesh (hardcoded!?)
        # so we will do the same here 
        rot = trimesh.transformations.rotation_matrix(np.radians(180), [1, 0, 0])
        mesh.apply_transform(rot)


        # material = pyrender.MetallicRoughnessMaterial(
        #     metallicFactor=0.0,
        #     alphaMode='OPAQUE',
        #     baseColorFactor=(0.65098039,  0.74117647,  0.85882353, 1.0)
        # )
        
        scene.add(pyrender.Mesh.from_trimesh(mesh, smooth=False))

        color, rend_depth = renderer.render(scene, flags=pyrender.RenderFlags.RGBA)

        
        img[color[:,:,3] > 0] = color[color[:,:,3] > 0, :3]

    
       

    #plt.show()
    #plt.savefig(f"out/{i}.png")
    # Save image again
    plt.imsave(f"out/{frame_id}.png", img)

    # if i > 5:
    #     break
    
    # i += 1

  0%|          | 0/100 [00:00<?, ?it/s]